In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
count = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        count +=1
        if count>5:
            break
    if count>5:
        break


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/.DS_Store
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1952_bacteria_4883.jpeg
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg


In [2]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.metrics import accuracy_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
class PneumoniaDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in ['NORMAL', 'PNEUMONIA']:
            class_dir = os.path.join(root_dir, label)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(0 if label == 'NORMAL' else 1)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


train_dataset = PneumoniaDataset(root_dir='/kaggle/input/chest-xray-pneumonia/chest_xray/train', transform = transform)
test_dataset = PneumoniaDataset(root_dir='/kaggle/input/chest-xray-pneumonia/chest_xray/test', transform = transform)
val_dataset = PneumoniaDataset(root_dir='/kaggle/input/chest-xray-pneumonia/chest_xray/val', transform = transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [5]:
try:
    from torchvision.models import resnet18, ResNet18_Weights
    weights = ResNet18_Weights.IMAGENET1K_V1
    model = resnet18(weights=weights)
except AttributeError:
    model = models.resnet18(pretrained=True)  # fallback for older torchvision

model.fc = nn.Linear(model.fc.in_features, 2)    # normal or pheumonal
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0

  for images, label in train_loader:
    images = images.to(device)
    labels = label.to(device) # Corrected typo here

    optimizer.zero_grad() # Corrected typo here

    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 183MB/s]


Epoch [1/10], Loss: 0.00710332978722508
Epoch [1/10], Loss: 0.009382475738876437
Epoch [1/10], Loss: 0.01144075155989524
Epoch [1/10], Loss: 0.013353339367848964
Epoch [1/10], Loss: 0.01779292990093582
Epoch [1/10], Loss: 0.019156625025842818
Epoch [1/10], Loss: 0.022077597814834923
Epoch [1/10], Loss: 0.02544228184076906
Epoch [1/10], Loss: 0.026506861751796278
Epoch [1/10], Loss: 0.029921140133237545
Epoch [1/10], Loss: 0.03238748617333137
Epoch [1/10], Loss: 0.033319293904158234
Epoch [1/10], Loss: 0.033541488455482785
Epoch [1/10], Loss: 0.03391280685740015
Epoch [1/10], Loss: 0.03477212473567278
Epoch [1/10], Loss: 0.03568358031427202
Epoch [1/10], Loss: 0.0371230878850068
Epoch [1/10], Loss: 0.0379659367805244
Epoch [1/10], Loss: 0.0386609411541304
Epoch [1/10], Loss: 0.039448850656762444
Epoch [1/10], Loss: 0.041635389456902545
Epoch [1/10], Loss: 0.042788326397446765
Epoch [1/10], Loss: 0.04332253106075562
Epoch [1/10], Loss: 0.04478986677856533
Epoch [1/10], Loss: 0.0449817320

In [6]:
model.eval()
val_labels = []
val_preds = []
with torch.no_grad():
  for images, labels in val_loader:
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    _, preds = torch.max(outputs, 1)

    val_labels.extend(labels.cpu().numpy())
    val_preds.extend(preds.cpu().numpy())

val_acc = accuracy_score(val_labels, val_preds)
print('validation accuracy:', val_acc)

validation accuracy: 1.0


In [7]:
model.eval()
test_labels = []
test_preds = []

with torch.no_grad():
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    _, preds = torch.max(outputs, 1)

    test_labels.extend(labels.cpu().numpy())
    test_preds.extend(preds.cpu().numpy())

test_acc = accuracy_score(val_labels, val_preds)
print('test accuracy:', test_acc)

test accuracy: 1.0


In [8]:
torch.save(model.state_dict(), 'pneumonia_classifier.pth')